# Import des packages

In [1]:
import duckdb
import pandas as pd

# Chemin et données brutes

In [2]:
path_to_read = "../data/data_phyto/raw"
path_to_save = "../data/data_phyto/processed"

In [24]:
data_epci = pd.read_csv(f"{path_to_read}/data_epci.csv", delimiter=';',encoding='latin1')
data_sau = duckdb.read_csv(f"{path_to_read}/sau_2025.csv")


In [25]:
data_epci

,dept,siren,raison_sociale,nature_juridique,mode_financ,nb_membres,total_pop_tot,total_pop_mun,dep_com,insee,siren_membre,nom_membre,ptot_2025,pmun_2025
0,01,200029999.0,CC Rives de l'Ain - Pays du Cerdon,CC,FPU,14.0,15 156,14 873,01,01304,210103040.0,Pont-d'Ain,2 912,2 862
1,01,200029999.0,CC Rives de l'Ain - Pays du Cerdon,CC,FPU,14.0,15 156,14 873,01,01199,210101994.0,Jujurieux,2 246,2 209
2,01,200029999.0,CC Rives de l'Ain - Pays du Cerdon,CC,FPU,14.0,15 156,14 873,01,01363,210103636.0,Saint-Jean-le-Vieux,1 873,1 799
3,01,200029999.0,CC Rives de l'Ain - Pays du Cerdon,CC,FPU,14.0,15 156,14 873,01,01314,210103149.0,Priay,1 826,1 803
4,01,200029999.0,CC Rives de l'Ain - Pays du Cerdon,CC,FPU,14.0,15 156,14 873,01,01273,210102737.0,Neuville-sur-Ain,1 823,1 798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34867,2B,242020105.0,CC de Calvi Balagne,CC,FPU,14.0,12 842,12 638,2B,2B138,212001382.0,Lavatoggio,158,156
34868,2B,242020105.0,CC de Calvi Balagne,CC,FPU,14.0,12 842,12 638,2B,2B296,212002968.0,Sant'Antonino,133,132
34869,2B,242020105.0,CC de Calvi Balagne,CC,FPU,14.0,12 842,12 638,2B,2B153,212001531.0,Manso,120,117
34870,2B,242020105.0,CC de Calvi Balagne,CC,FPU,14.0,12 842,12 638,2B,2B025,212000251.0,Avapessa,81,81


In [26]:
#filtre data_epci
query = """ 
 SELECT
    DISTINCT siren AS id_epci,
    raison_sociale AS nom_epci,
    dept
 FROM data_epci
 """

df_epci = duckdb.sql(query)

## Fichier phyto 

### Phyto par commune avec sau

In [5]:
data_sau

┌─────────────────────┬──────────────┬──────────────────────────────────────────────────────────────┬────────────────────┐
│     date_mesure     │ geocode_epci │                         libelle_epci                         │       valeur       │
│      timestamp      │   varchar    │                           varchar                            │       double       │
├─────────────────────┼──────────────┼──────────────────────────────────────────────────────────────┼────────────────────┤
│ 2010-01-01 00:00:00 │ 200066041    │ Communauté de communes Sundgau                               │           21040.35 │
│ 2020-01-01 00:00:00 │ 247400773    │ Communauté de communes des Sources du Lac d'Annecy           │ 2228.3399999999997 │
│ 2020-01-01 00:00:00 │ 245501176    │ Communauté de communes du Territoire de Fresnes-en-Woëvre    │ 15404.110000000002 │
│ 2010-01-01 00:00:00 │ 243800984    │ Communauté d'agglomération du Pays Voironnais                │ 14426.220000000003 │
│ 2010-01-01 00:

In [8]:
query = """ 
 SELECT
    geocode_epci,
    ROUND(TRY_CAST(valeur AS DOUBLE), 2) AS sau_ha
FROM data_sau
WHERE geocode_epci NOT LIKE 'Z%' and date_mesure >= '2020-01-01'
"""

data_sau = duckdb.sql(query)
data_sau

┌──────────────┬──────────┐
│ geocode_epci │  sau_ha  │
│   varchar    │  double  │
├──────────────┼──────────┤
│ 247400773    │  2228.34 │
│ 245501176    │ 15404.11 │
│ 200066785    │ 37332.64 │
│ 200040574    │  8968.44 │
│ 242010130    │  9759.43 │
│ 200071926    │ 27862.03 │
│ 200070670    │ 22086.67 │
│ 200033025    │ 16748.35 │
│ 200040798    │  3634.74 │
│ 247000664    │   5774.7 │
│     ·        │      ·   │
│     ·        │      ·   │
│     ·        │      ·   │
│ 200067767    │  9920.79 │
│ 200041762    │ 23892.37 │
│ 200058477    │   210.75 │
│ 200041861    │  19108.4 │
│ 200067064    │  7862.31 │
│ 240800862    │ 62670.81 │
│ 244301131    │   4687.1 │
│ 249500489    │  1769.72 │
│ 246700967    │  8109.07 │
│ 200041168    │  10541.1 │
├──────────────┴──────────┤
│  1255 rows (20 shown)   │
└─────────────────────────┘

In [9]:
data_phyto = duckdb.read_parquet(f"{path_to_read}/achat_commune_phyto.parquet")

In [28]:
data_phyto

┌─────────┬────────────────────────────┬──────────────────────┬──────────────────────────┬───────────────────────────┬────────────┬─────────────────────┬─────────┬─────────────┬─────────┬────────────────────────────────┬─────────────┬───────────────────────────────────────────────────────────────────────────────────────┬─────────┬────────────────────┐
│  annee  │    nom_region_acheteur     │ code_region_acheteur │ nom_departement_acheteur │ code_departement_acheteur │ code_insee │    libelle_insee    │   amm   │ biocontrole │   uab   │           substance            │     cas     │                                     fonction_sdes                                     │ danger  │ quantite_substance │
│ varchar │          varchar           │       varchar        │         varchar          │          varchar          │  varchar   │       varchar       │ varchar │   varchar   │ varchar │            varchar             │   varchar   │                                        varchar           

In [10]:
query_phyto = """
SELECT 
    annee,
    TRY_CAST(quantite_substance AS DOUBLE) AS quantite_substance,
    code_insee
FROM data_phyto
"""

phyto = duckdb.sql(query_phyto)
phyto

┌─────────┬────────────────────┬────────────┐
│  annee  │ quantite_substance │ code_insee │
│ varchar │       double       │  varchar   │
├─────────┼────────────────────┼────────────┤
│ 2015    │              3.789 │ 47247      │
│ 2015    │              0.002 │ 47058      │
│ 2015    │              0.153 │ 47177      │
│ 2015    │              0.078 │ 47250      │
│ 2015    │              0.253 │ 47251      │
│ 2015    │                0.0 │ 33155      │
│ 2015    │               0.25 │ 47128      │
│ 2015    │            137.247 │ 47071      │
│ 2015    │              0.107 │ 47174      │
│ 2015    │              0.033 │ 47197      │
│  ·      │                ·   │   ·        │
│  ·      │                ·   │   ·        │
│  ·      │                ·   │   ·        │
│ 2015    │              0.003 │ 04076      │
│ 2015    │                0.0 │ 06047      │
│ 2015    │              0.253 │ 06158      │
│ 2015    │              0.001 │ 06015      │
│ 2015    │              0.004 │ 0

In [11]:
#jointure avec epci
query_merged_phyto = """ 
SELECT
    phyto.annee,
    TRY_CAST(phyto.quantite_substance AS DOUBLE) AS quantite_substance,
    epci.siren
FROM data_phyto AS phyto
INNER JOIN data_epci AS epci
    ON epci.insee = phyto.code_insee
"""

merged_phyto = duckdb.sql(query_merged_phyto)


In [12]:
merged_phyto

┌─────────┬────────────────────┬───────────┐
│  annee  │ quantite_substance │   siren   │
│ varchar │       double       │   int64   │
├─────────┼────────────────────┼───────────┤
│ 2015    │              3.789 │ 244700449 │
│ 2015    │              0.002 │ 244701355 │
│ 2015    │              0.153 │ 200068922 │
│ 2015    │              0.078 │ 200068922 │
│ 2015    │              0.253 │ 200068922 │
│ 2015    │                0.0 │ 200043982 │
│ 2015    │               0.25 │ 200096956 │
│ 2015    │            137.247 │ 244701405 │
│ 2015    │              0.107 │ 200068948 │
│ 2015    │              0.033 │ 200068948 │
│  ·      │                ·   │     ·     │
│  ·      │                ·   │     ·     │
│  ·      │                ·   │     ·     │
│ 2015    │              0.879 │ 200023919 │
│ 2015    │              0.516 │ 247700107 │
│ 2015    │              0.413 │ 247700032 │
│ 2015    │              2.931 │ 200023240 │
│ 2015    │              0.168 │ 200023240 │
│ 2015    

In [13]:
query = """ 
WITH df_temp AS (
    SELECT
        siren,
        COUNT(DISTINCT annee) AS n_years,
        SUM(quantite_substance) AS total_quantite_substance
    FROM merged_phyto
    GROUP BY siren
)

SELECT
    tp.siren,
    (1.0*tp.total_quantite_substance / tp.n_years) AS avg_annual_quantite_substance
FROM df_temp AS tp
ORDER BY tp.siren
"""

avg_annual_phyto = duckdb.sql(query)
avg_annual_phyto

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌───────────┬───────────────────────────────┐
│   siren   │ avg_annual_quantite_substance │
│   int64   │            double             │
├───────────┼───────────────────────────────┤
│ 200000172 │             999.4994444444435 │
│ 200000438 │             8053.641222222251 │
│ 200000545 │             19412.45077777776 │
│ 200000628 │            37880.982555555514 │
│ 200000800 │             4523.816666666687 │
│ 200000925 │            31344.158888888494 │
│ 200000933 │             71680.71677777727 │
│ 200004802 │            2637.5911111111254 │
│ 200005932 │             5736.507222222239 │
│ 200005957 │            10099.099666666938 │
│     ·     │                     ·         │
│     ·     │                     ·         │
│     ·     │                     ·         │
│ 249720053 │            1515.4901111111112 │
│ 249720061 │             737.1668333333333 │
│ 249730037 │            268.44666666666666 │
│ 249730045 │             673.2446666666666 │
│ 249730052 │                     

In [14]:

df_sau = pd.read_csv("https://www.data.gouv.fr/api/1/datasets/r/022cb00f-38f2-4fe7-8895-e3467d3d9255", sep=",")

In [15]:
df_sau

,date_mesure,geocode_epci,libelle_epci,valeur
0,2010-01-01T00:00:00.000,200066041,Communauté de communes Sundgau,21040.35
1,2020-01-01T00:00:00.000,247400773,Communauté de communes des Sources du Lac d'An...,2228.34
2,2020-01-01T00:00:00.000,245501176,Communauté de communes du Territoire de Fresne...,15404.11
3,2010-01-01T00:00:00.000,243800984,Communauté d'agglomération du Pays Voironnais,14426.22
4,2010-01-01T00:00:00.000,200073252,Communauté de communes de la Castagniccia-Casinca,8354.97
...,...,...,...,...
2502,2020-01-01T00:00:00.000,246700967,Communauté de communes de Sélestat,8109.07
2503,2010-01-01T00:00:00.000,200072635,Communauté de communes Pyrénées Haut Garonnaises,7747.39
2504,2010-01-01T00:00:00.000,200041960,Communauté de communes Pévèle-Carembault,19151.62
2505,2020-01-01T00:00:00.000,200041168,Communauté de communes de la Vallée de l'Homme,10541.10


In [16]:
query_sau = """ 
SELECT geocode_epci, ROUND(TRY_CAST(valeur AS DOUBLE), 2) AS sau_ha
FROM df_sau
WHERE geocode_epci NOT LIKE 'Z%' AND date_mesure LIKE '2020%'
"""

df_sau = duckdb.sql(query_sau)
df_sau

┌──────────────┬──────────┐
│ geocode_epci │  sau_ha  │
│   varchar    │  double  │
├──────────────┼──────────┤
│ 247400773    │  2228.34 │
│ 245501176    │ 15404.11 │
│ 200066785    │ 37332.64 │
│ 200040574    │  8968.44 │
│ 242010130    │  9759.43 │
│ 200071926    │ 27862.03 │
│ 200070670    │ 22086.67 │
│ 200033025    │ 16748.35 │
│ 200040798    │  3634.74 │
│ 247000664    │   5774.7 │
│     ·        │      ·   │
│     ·        │      ·   │
│     ·        │      ·   │
│ 200067767    │  9920.79 │
│ 200041762    │ 23892.37 │
│ 200058477    │   210.75 │
│ 200041861    │  19108.4 │
│ 200067064    │  7862.31 │
│ 240800862    │ 62670.81 │
│ 244301131    │   4687.1 │
│ 249500489    │  1769.72 │
│ 246700967    │  8109.07 │
│ 200041168    │  10541.1 │
├──────────────┴──────────┤
│  1255 rows (20 shown)   │
└─────────────────────────┘

In [17]:
# On ramène to ça à la surface agricole utile
query_final = """
SELECT
    aap.siren,
    ROUND((1.0 * aap.avg_annual_quantite_substance / ds.sau_ha), 4) AS phyto_per_ha_sau
FROM avg_annual_phyto AS aap
INNER JOIN data_sau AS ds
    ON aap.siren = ds.geocode_epci
""" 

final_phyto = duckdb.sql(query_final)
final_phyto

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌───────────┬──────────────────┐
│   siren   │ phyto_per_ha_sau │
│   int64   │      double      │
├───────────┼──────────────────┤
│ 200071876 │           4.3285 │
│ 200071884 │           0.2051 │
│ 200071892 │           4.2574 │
│ 200071900 │           1.1436 │
│ 200071918 │           1.6156 │
│ 200071926 │           6.9035 │
│ 200071967 │           0.2186 │
│ 200071983 │           2.8004 │
│ 200071991 │            4.621 │
│ 200072007 │           0.0209 │
│     ·     │              ·   │
│     ·     │              ·   │
│     ·     │              ·   │
│ 200034692 │          17.0866 │
│ 200069821 │           3.4976 │
│ 200071074 │           2.4625 │
│ 200071629 │            0.852 │
│ 200006930 │           0.0233 │
│ 200035855 │           2.1536 │
│ 200067940 │           0.0826 │
│ 200070944 │           4.2422 │
│ 200069300 │           0.9857 │
│ 200023778 │           0.9309 │
├───────────┴──────────────────┤
│     1254 rows (20 shown)     │
└──────────────────────────────┘

In [27]:
query_final = """ 
SELECT
    df_epci.dept,
    df_epci.id_epci,
    df_epci.nom_epci,
    'i1140' AS id_indicator,
    final_phyto.phyto_per_ha_sau AS valeur_brute
FROM df_epci
LEFT JOIN final_phyto
    ON df_epci.id_epci = final_phyto.siren
ORDER BY df_epci.dept, df_epci.id_epci
    """

df_final = duckdb.sql(query_final)
df_final

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌─────────┬─────────────┬─────────────────────────────────────────────────┬──────────────┬──────────────┐
│  dept   │   id_epci   │                    nom_epci                     │ id_indicator │ valeur_brute │
│ varchar │   double    │                     varchar                     │   varchar    │    double    │
├─────────┼─────────────┼─────────────────────────────────────────────────┼──────────────┼──────────────┤
│ 01      │ 200029999.0 │ CC Rives de l'Ain - Pays du Cerdon              │ i1140        │       2.0024 │
│ 01      │ 200040350.0 │ CC Bugey Sud                                    │ i1140        │       0.9376 │
│ 01      │ 200042497.0 │ CC Dombes Saône Vallée                          │ i1140        │       1.9112 │
│ 01      │ 200042935.0 │ CA Haut-Bugey Agglomération                     │ i1140        │       0.1537 │
│ 01      │ 200069193.0 │ CC de la Dombes                                 │ i1140        │       1.5883 │
│ 01      │ 200070118.0 │ CC Val de Saône Cent

In [29]:
path_to_save = "../data/data_phyto/processed"
df_final.write_csv(f"{path_to_save}/i114_kg_phyto_per_ha_sau.csv")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

### Phytorelative par epci

In [ ]:
#Requête pour obtenir la quantité totale de substance par département, par année et par EPCI
#Requête lourde, à optimiser plus tard si besoin
q_phyto_per_year = """ 
SELECT 
    annee,
    dept,
    siren,
    SUM(quantite_substance) AS total_quantite_substance
FROM phyto
GROUP BY annee, dept, siren
ORDER BY dept, annee, siren
"""

phyto_per_year = duckdb.sql(q_phyto_per_year)
phyto_per_year

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

IOException: IO Error: Could not write file ".tmp/duckdb_temp_block-4611686018427494457.block": No space left on device

In [8]:
data_phyto_relative = duckdb.read_csv(f"{path_to_save}/phyto_relative_csv.csv")
data_phyto_relative


┌───────────┬─────────┬─────────┬───────────────┐
│   siren   │ pop_tot │ dep_com │ total_relatif │
│   int64   │ varchar │ varchar │    double     │
├───────────┼─────────┼─────────┼───────────────┤
│ 200070852 │ 21 965  │ 01      │          1.06 │
│ 200070118 │ 21 470  │ 01      │          1.81 │
│ 240100750 │ 106 924 │ 01      │          0.89 │
│ 240100883 │ 84 625  │ 01      │          4.32 │
│ 200040590 │ 75 051  │ 01      │          6.09 │
│ 240100800 │ 25 583  │ 01      │           1.0 │
│ 200029999 │ 15 156  │ 01      │          0.96 │
│ 200042497 │ 41 975  │ 01      │          1.94 │
│ 200042935 │ 65 428  │ 01      │          0.21 │
│ 200069193 │ 41 461  │ 01      │          5.48 │
│     ·     │   ·     │ ·       │            ·  │
│     ·     │   ·     │ ·       │            ·  │
│     ·     │   ·     │ ·       │            ·  │
│ 249730045 │ 153 249 │ 973     │          0.07 │
│ 249740077 │ 185 848 │ 974     │          1.29 │
│ 249740101 │ 221 532 │ 974     │          0.32 │


In [17]:
q_phyto_relative = """ 
SELECT
    p2.dept,
    p1.siren,
    TRY_CAST(REPLACE(p1.pop_tot,' ','') AS DOUBLE) AS pop_tot,
    p1.total_relatif AS total_relatif,
    p2.total_sau2020
FROM data_phyto_relative AS p1
INNER JOIN df_sau_totale AS p2 ON p1.siren = p2.siren
ORDER BY p2.dept, p1.siren
"""

df_phyto_relative = duckdb.sql(q_phyto_relative)
df_phyto_relative.to_csv(f"{path_to_save}/phyto_relative_csv.csv")